# The `xarray` package for multidimensional data

Notes from [Multi-Dimensional Arrays for Decision Analysis, Persuasive Python](https://www.persuasivepython.com/4-multidimarrays)

Here we'll experiment with the `xarray` package as a method of storing multiple simulation runs, as a more flexible alternative to dataframes and more user-friendly than nested dictionaries

Concepts:

* `DataArray`s
* Naming dimensions with `dims`
* Coordinates index an array and are specified with `coords`
* A `DataSet` can be created with multiple `DataArray`s

This example, we'll simulate demand values which takes on 100 draws from a binomial distribution, or:

$$
d \sim Binomial(200, 0.2)
$$

In [76]:
from numpy.random import default_rng
import numpy as np
import xarray as xr

rng = default_rng(seed = 111)  ## set random seed 
demand = rng.binomial(n=200,p=0.2,size=100)   ## get demand values

## make data array
xr.DataArray(data=demand)

<xarray.DataArray (dim_0: 100)>
array([42, 47, 43, 41, 38, 40, 41, 45, 29, 29, 49, 44, 41, 46, 40, 39, 37,
       41, 37, 37, 38, 28, 28, 39, 48, 38, 47, 41, 43, 37, 40, 41, 35, 44,
       41, 35, 35, 39, 35, 47, 45, 35, 30, 48, 44, 38, 35, 40, 44, 41, 38,
       47, 42, 35, 36, 36, 44, 37, 42, 41, 40, 36, 36, 43, 43, 48, 27, 44,
       38, 37, 32, 48, 49, 29, 37, 44, 46, 42, 33, 36, 49, 33, 46, 35, 36,
       34, 45, 36, 41, 44, 35, 36, 38, 31, 50, 47, 43, 47, 31, 37])
Dimensions without coordinates: dim_0

### Naming Dimensions

We'll instead give this data array a meaningful name for the dimension instead of `dim_0`

In [77]:
xr.DataArray(data=demand, 
             dims='draw')

<xarray.DataArray (draw: 100)>
array([42, 47, 43, 41, 38, 40, 41, 45, 29, 29, 49, 44, 41, 46, 40, 39, 37,
       41, 37, 37, 38, 28, 28, 39, 48, 38, 47, 41, 43, 37, 40, 41, 35, 44,
       41, 35, 35, 39, 35, 47, 45, 35, 30, 48, 44, 38, 35, 40, 44, 41, 38,
       47, 42, 35, 36, 36, 44, 37, 42, 41, 40, 36, 36, 43, 43, 48, 27, 44,
       38, 37, 32, 48, 49, 29, 37, 44, 46, 42, 33, 36, 49, 33, 46, 35, 36,
       34, 45, 36, 41, 44, 35, 36, 38, 31, 50, 47, 43, 47, 31, 37])
Dimensions without coordinates: draw

### Coordinates

A `coordinate` is used to index an array. Can specify the `coordinate` with the `coords` argument and a dictionary:

In [78]:
demandDA = xr.DataArray(data=demand,
                        coords={'draw':np.arange(100)+1},
                        name='demand')
demandDA

<xarray.DataArray 'demand' (draw: 100)>
array([42, 47, 43, 41, 38, 40, 41, 45, 29, 29, 49, 44, 41, 46, 40, 39, 37,
       41, 37, 37, 38, 28, 28, 39, 48, 38, 47, 41, 43, 37, 40, 41, 35, 44,
       41, 35, 35, 39, 35, 47, 45, 35, 30, 48, 44, 38, 35, 40, 44, 41, 38,
       47, 42, 35, 36, 36, 44, 37, 42, 41, 40, 36, 36, 43, 43, 48, 27, 44,
       38, 37, 32, 48, 49, 29, 37, 44, 46, 42, 33, 36, 49, 33, 46, 35, 36,
       34, 45, 36, 41, 44, 35, 36, 38, 31, 50, 47, 43, 47, 31, 37])
Coordinates:
  * draw     (draw) int64 1 2 3 4 5 6 7 8 9 10 ... 92 93 94 95 96 97 98 99 100

Now we have an array of demand values $d_i$ where $i$ is a specific coordinate.

In [79]:
## creating a DataArray of order quantities - must use name now to create dataset later
orderDA = xr.DataArray(data = np.arange(25,51), 
                       coords = {"orderQtyIndex": np.arange(25,51)},
                       name = "orderQty")
orderDA

<xarray.DataArray 'orderQty' (orderQtyIndex: 26)>
array([25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41,
       42, 43, 44, 45, 46, 47, 48, 49, 50])
Coordinates:
  * orderQtyIndex  (orderQtyIndex) int64 25 26 27 28 29 30 ... 45 46 47 48 49 50

## Creating a `Dataset` from multiple `DataArray` objects

Here's the magic

In [80]:
# create dataset by combining data arrays
ds = xr.merge([demandDA,orderDA])
ds

<xarray.Dataset>
Dimensions:        (draw: 100, orderQtyIndex: 26)
Coordinates:
  * draw           (draw) int64 1 2 3 4 5 6 7 8 9 ... 93 94 95 96 97 98 99 100
  * orderQtyIndex  (orderQtyIndex) int64 25 26 27 28 29 30 ... 45 46 47 48 49 50
Data variables:
    demand         (draw) int64 42 47 43 41 38 40 41 45 ... 31 50 47 43 47 31 37
    orderQty       (orderQtyIndex) int64 25 26 27 28 29 30 ... 45 46 47 48 49 50

Now we can create new data variables using the `.assign` method which you can use to calculate across data variables.

In the example below, we'll create a new data variable `soldNewsPapers` which is the minimum of `orderQty` and `demand` DataArrays:

In [81]:
ds = (ds
          .assign(soldNewspapers = np.minimum(ds.demand,ds.orderQty))
)
ds

<xarray.Dataset>
Dimensions:         (draw: 100, orderQtyIndex: 26)
Coordinates:
  * draw            (draw) int64 1 2 3 4 5 6 7 8 9 ... 93 94 95 96 97 98 99 100
  * orderQtyIndex   (orderQtyIndex) int64 25 26 27 28 29 30 ... 46 47 48 49 50
Data variables:
    demand          (draw) int64 42 47 43 41 38 40 41 ... 31 50 47 43 47 31 37
    orderQty        (orderQtyIndex) int64 25 26 27 28 29 30 ... 46 47 48 49 50
    soldNewspapers  (draw, orderQtyIndex) int64 25 26 27 28 29 ... 37 37 37 37

But didn't `demand` have 100 draws and `orderQty` have 26 elements? Let's take a closer inspection of the result:

In [82]:
ds.soldNewspapers

<xarray.DataArray 'soldNewspapers' (draw: 100, orderQtyIndex: 26)>
array([[25, 26, 27, ..., 42, 42, 42],
       [25, 26, 27, ..., 47, 47, 47],
       [25, 26, 27, ..., 43, 43, 43],
       ...,
       [25, 26, 27, ..., 47, 47, 47],
       [25, 26, 27, ..., 31, 31, 31],
       [25, 26, 27, ..., 37, 37, 37]])
Coordinates:
  * draw           (draw) int64 1 2 3 4 5 6 7 8 9 ... 93 94 95 96 97 98 99 100
  * orderQtyIndex  (orderQtyIndex) int64 25 26 27 28 29 30 ... 45 46 47 48 49 50

We now have a shape (100, 26) data array, what's in each element?

In [83]:
ds.soldNewspapers[0]

<xarray.DataArray 'soldNewspapers' (orderQtyIndex: 26)>
array([25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41,
       42, 42, 42, 42, 42, 42, 42, 42, 42])
Coordinates:
    draw           int64 1
  * orderQtyIndex  (orderQtyIndex) int64 25 26 27 28 29 30 ... 45 46 47 48 49 50

What's neat here is that we created a new 100 x 26 DataArray, where each element in the dataarray is a single draw of demand (from a binomial distribution). It's like a mega cross-join!

It might be useful to see this as a dataframe:

In [84]:
ds.to_dataframe()

demand  orderQty  soldNewspapers
draw orderQtyIndex                                  
1    25                 42        25              25
     26                 42        26              26
     27                 42        27              27
     28                 42        28              28
     29                 42        29              29
...                    ...       ...             ...
100  46                 37        46              37
     47                 37        47              37
     48                 37        48              37
     49                 37        49              37
     50                 37        50              37

[2600 rows x 3 columns]

Adding a more complex function using a `lambda` function:

In [85]:
(  
    ds
    .assign(soldNewspapers = np.minimum(ds.demand,ds.orderQty))
    .assign(revenue = lambda DS: 3 * DS.soldNewspapers)
)

<xarray.Dataset>
Dimensions:         (draw: 100, orderQtyIndex: 26)
Coordinates:
  * draw            (draw) int64 1 2 3 4 5 6 7 8 9 ... 93 94 95 96 97 98 99 100
  * orderQtyIndex   (orderQtyIndex) int64 25 26 27 28 29 30 ... 46 47 48 49 50
Data variables:
    demand          (draw) int64 42 47 43 41 38 40 41 ... 31 50 47 43 47 31 37
    orderQty        (orderQtyIndex) int64 25 26 27 28 29 30 ... 46 47 48 49 50
    soldNewspapers  (draw, orderQtyIndex) int64 25 26 27 28 29 ... 37 37 37 37
    revenue         (draw, orderQtyIndex) int64 75 78 81 84 ... 111 111 111 111

Extra practice, instead of having `price=3`, we can test different prices. We must first add in a new `DataArray` 

In [86]:
## creating a DataArray of Prices - must use name now to create dataset later
priceDA = xr.DataArray(data = np.arange(2,6), 
                       coords = {"priceIndex": np.arange(2,6)},
                       name = "price")
priceDA

<xarray.DataArray 'price' (priceIndex: 4)>
array([2, 3, 4, 5])
Coordinates:
  * priceIndex  (priceIndex) int64 2 3 4 5

In [88]:
ds2 = (ds
       .merge(priceDA)
       .assign(soldNewspapers = np.minimum(ds.demand,ds.orderQty))
       .assign(revenue=lambda x: x.price * x.soldNewspapers)
       )
ds2

<xarray.Dataset>
Dimensions:         (draw: 100, orderQtyIndex: 26, priceIndex: 4)
Coordinates:
  * draw            (draw) int64 1 2 3 4 5 6 7 8 9 ... 93 94 95 96 97 98 99 100
  * orderQtyIndex   (orderQtyIndex) int64 25 26 27 28 29 30 ... 46 47 48 49 50
  * priceIndex      (priceIndex) int64 2 3 4 5
Data variables:
    demand          (draw) int64 42 47 43 41 38 40 41 ... 31 50 47 43 47 31 37
    orderQty        (orderQtyIndex) int64 25 26 27 28 29 30 ... 46 47 48 49 50
    soldNewspapers  (draw, orderQtyIndex) int64 25 26 27 28 29 ... 37 37 37 37
    price           (priceIndex) int64 2 3 4 5
    revenue         (priceIndex, draw, orderQtyIndex) int64 50 52 54 ... 185 185

## Subsetting

Let's look at the situation where we ordered 36 newspapers:

In [89]:
ds2.sel(orderQtyIndex=36)

<xarray.Dataset>
Dimensions:         (draw: 100, priceIndex: 4)
Coordinates:
  * draw            (draw) int64 1 2 3 4 5 6 7 8 9 ... 93 94 95 96 97 98 99 100
    orderQtyIndex   int64 36
  * priceIndex      (priceIndex) int64 2 3 4 5
Data variables:
    demand          (draw) int64 42 47 43 41 38 40 41 ... 31 50 47 43 47 31 37
    orderQty        int64 36
    soldNewspapers  (draw) int64 36 36 36 36 36 36 36 ... 31 36 36 36 36 31 36
    price           (priceIndex) int64 2 3 4 5
    revenue         (priceIndex, draw) int64 72 72 72 72 72 ... 180 180 155 180

We have every draw of demand here, and we can also see the `soldNewspapers` in this context:

In [90]:
ds2.sel(orderQtyIndex=36).soldNewspapers

<xarray.DataArray 'soldNewspapers' (draw: 100)>
array([36, 36, 36, 36, 36, 36, 36, 36, 29, 29, 36, 36, 36, 36, 36, 36, 36,
       36, 36, 36, 36, 28, 28, 36, 36, 36, 36, 36, 36, 36, 36, 36, 35, 36,
       36, 35, 35, 36, 35, 36, 36, 35, 30, 36, 36, 36, 35, 36, 36, 36, 36,
       36, 36, 35, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 27, 36,
       36, 36, 32, 36, 36, 29, 36, 36, 36, 36, 33, 36, 36, 33, 36, 35, 36,
       34, 36, 36, 36, 36, 35, 36, 36, 31, 36, 36, 36, 36, 31, 36])
Coordinates:
  * draw           (draw) int64 1 2 3 4 5 6 7 8 9 ... 93 94 95 96 97 98 99 100
    orderQtyIndex  int64 36

Or look at the results when `price=3` - how about revenue for each value of order quantity in a dataframe:

In [98]:
(ds2.sel(priceIndex=3)
    .revenue
    .sum(axis=0)
    .to_dataframe())

,priceIndex,revenue
orderQtyIndex,,
25,3,7500
26,3,7800
27,3,8100
28,3,8397
29,3,8688
30,3,8970
31,3,9249
32,3,9522
33,3,9792
